In [1]:
from gpiozero import RotaryEncoder, Button
from time import sleep
import threading
from TGS2620Sensor import TGS2620Sensor
from MQ2Sensor import MQ2Sensor

In [2]:
encoder = RotaryEncoder(a=6, b=5)  # CLK连接GPIO6，DT连接GPIO5
button = Button(12)                  # SW连接 GPIO12
globalCounter = 0          # 计数器值
tmp = 0	                # 当前状态判断

In [3]:
# 模式列表
TGS2620 = TGS2620Sensor()
MQ2 = MQ2Sensor()
modes = ["MPU6050", "MAX30102", "MQ2", "TGS2620", "Start All", "Shut Down All"]
start_modes = ["Start MPU6050", "Start MAX30102", "Start MQ2", "Start TGS2620", "Start All", "Shut Down All"]
run_modes = [None, None, MQ2.run, TGS2620.run, None, None]
stop_modes = [None, None, MQ2.stop, TGS2620.stop, None, None]
threads = {}

In [4]:
# 旋转编码方向位判断函数
def rotaryDeal():
    global globalCounter
    global tmp
    globalCounter += encoder.steps  # 根据编码器步骤调整计数器
    encoder.steps = 0 # 更新计数器后重置编码器步骤
    if tmp != globalCounter: # 判断状态值发生改变
        mode_index = globalCounter % len(modes)
        print (f"Mode: {modes[mode_index]}") # 打印出状态信息
        tmp = globalCounter    #  把当前状态赋值到下一个状态，避免重复打印

# 中间按键按下响应程序
def btnISR():
    mode_index = globalCounter % len(start_modes)
    print(f"Confirm Mode: {start_modes[mode_index]}")

    if run_modes[mode_index] and stop_modes[mode_index]:
        if mode_index not in threads or not threads[mode_index].is_alive():
            thread = threading.Thread(target=run_modes[mode_index], daemon=True)
            threads[mode_index] = thread
            thread.start()
            print(start_modes[mode_index])
        else:
            print(f"{modes[mode_index]} is already running")
            stop_modes[mode_index]()
            threads[mode_index].join()
            print(f"{modes[mode_index]} killed")
        
    sleep(1)  # 防抖

# 按键按下中断响应程序
button.when_pressed = btnISR

In [5]:
if __name__ == '__main__':
    try:
        while True:
            rotaryDeal()  # 更新编码器值
            sleep(0.01)             # 短延迟，降低CPU负载
    except KeyboardInterrupt:
        pass

Mode: MAX30102
Mode: MQ2
Confirm Mode: Start MQ2
MQ2 run

Gas Value: 9
Initial volume: 0.03370786516853941
Start MQ2
Gas Value: 9
Initial volume: 0.03370786516853941
Gas Value: 9
Initial volume: 0.03370786516853941
Gas Value: 9
Initial volume: 0.03370786516853941
Mode: TGS2620
Gas Value: 9
Initial volume: 0.03370786516853941
Gas Value: 9
Initial volume: 0.03370786516853941
Gas Value: 9
Initial volume: 0.03468490473864194
Confirm Mode: Start TGS2620
Alcohol Value: 117
Start TGS2620
Gas Value: 9
Initial volume: 0.03370786516853941
Alcohol Value: 77
Gas Value: 9
Initial volume: 0.03468490473864194
Alcohol Value: 23
Gas Value: 9
Initial volume: 0.03370786516853941
Alcohol Value: 10
Gas Value: 9
Initial volume: 0.03370786516853941
Alcohol Value: 0
Gas Value: 9
Initial volume: 0.03370786516853941
Alcohol Value: 3
Gas Value: 9
Initial volume: 0.03370786516853941
Alcohol Value: 6
Gas Value: 8
Initial volume: 0.03273082559843665
Alcohol Value: 7
Gas Value: 9
Initial volume: 0.03370786516853941
